# NASA ABoVE Arctic Lakes ASD Data Processing
### Testing for Optical and Biogeochemical Relationships

**Authors:** Catherine Kuhn   
**Last Updated:** Nov 27th, 2018

In [1]:
### Import the required python libraries
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import glob
import os
import sys
import csv
from scipy import interpolate
from os import listdir
from os.path import isfile, join
import matplotlib.gridspec as gridspec
#from pylr2 import regress2
import scipy.stats as stats
import seaborn as sns
import math

In [2]:
%cd /Users/ckuhn/Downloads
df = pd.read_csv('processed_Data.csv', low_memory=False)

/Users/ckuhn/Downloads


In [3]:
df.head(3)

,rep_name,wavelength,asd_measure,std_deviation,lake_name
0,sky,350,1.663184,1.110763,9mile
1,sky,351,1.654474,1.110803,9mile
2,sky,352,1.646593,1.112029,9mile


In [4]:
df['rep_name'].unique()

array(['sky', 'water', 'wr'], dtype=object)

### Calculate downwelling irradiance

Downwelling irrandiance ($E_d(\lambda)$) is calculated as: 

$E_d(\lambda)= L_{ref}(\lambda) * f_c * \pi  $  

where: 

- $E_d(\lambda)$ = downwelling irradiance   
- $L_{ref}(\lambda)$ = reference panel radiance  (wr measurement)  
- $f_c$ = correction factor ratio of field to lab radiance for Spectralon panel   

In [5]:
lakes = list(df['lake_name'].unique())
wavelengths = list(df['wavelength'].unique())

ed = []
lref = []
lake_name = []
wls = []
lsw = []
lsky = []
    
for lake in lakes:
    lake_df = df.loc[df['lake_name']==lake]
    for wavelength in wavelengths:
        new_df = lake_df.loc[lake_df['wavelength']==wavelength]
        sky_df = new_df.loc[new_df['rep_name']=='sky']            
        LSKY = sky_df['asd_measure'].item()
        reference_df = new_df.loc[new_df['rep_name']=='wr']            
        ref = reference_df['asd_measure'].item()
        downwelling_irradiance = ref #* 1 #* str(math.pi)
        print(downwelling_irradiance)

0.634608731
0.633766276
0.6327393
0.631907515
0.6310601370000001
0.630079426
0.62930036
0.628456408
0.62758505
0.626764225
0.6258695120000001
0.62516665
0.62445747
0.62347861
0.622890181
0.622243895
0.6215142770000001
0.621037696
0.620483393
0.619837385
0.6191374829999999
0.618429115
0.617799976
0.617158965
0.616445255
0.615801186
0.6151413579999999
0.614512202
0.613979194
0.613384025
0.612864938
0.6123556920000001
0.611644772
0.611006154
0.610288626
0.609514236
0.609029435
0.608547749
0.608029109
0.607524879
0.607161659
0.606712115
0.606180797
0.605662117
0.605134093
0.604651654
0.604201534
0.6037289
0.603339669
0.6030134189999999
0.602704989
0.6023642060000001
0.601980237
0.601584508
0.60120706
0.600855171
0.6004647320000001
0.60005787
0.5996773360000001
0.5992425029999999
0.59878831
0.598351116
0.597951844
0.597506849
0.597071695
0.5966779839999999
0.5962671429999999
0.595908449
0.595579147
0.5952217129999999
0.594757055
0.594334107
0.593941971
0.5934827779999999
0.593083255
0.59274

0.5642452370000001
0.563822527
0.563135938
0.562919872
0.562943944
0.562779358
0.563103618
0.562790046
0.561519953
0.56103527
0.560750414
0.560425551
0.560762277
0.560751509
0.560394876
0.560139417
0.560122555
0.5597780729999999
0.558731549
0.558120278
0.557181375
0.556892935
0.5599976839999999
0.5604995979999999
0.55909977
0.556011908
0.554262103
0.545301778
0.545196146
0.561800634
0.56948967
0.578392985
0.585828305
0.5779692120000001
0.585670618
0.62258375
0.612039191
0.569369425
0.5560259129999999
0.5080315089999999
0.505730141
0.582732719
0.593334876
0.552466221
0.548138763
0.53380574
0.531737654
0.5612734079999999
0.569372466
0.560772629
0.560701275
0.557917262
0.556441283
0.549022845
0.5640526260000001
0.563400232
0.560552642
0.556506057
0.5527616129999999
0.551366447
0.549442574
0.547677114
0.547623917
0.545972727
0.539234685
0.54294872
0.555504794
0.554553568
0.551231801
0.551499844
0.544766515
0.545537211
0.5560550129999999
0.557256505
0.554345577
0.5534474739999999
0.54998812

0.912256228
0.912185136
0.912135722
0.912119758
0.91215514
0.912229084
0.912257428
0.91231985
0.9124028609999999
0.9124503690000001
0.91240566
0.912365742
0.912376217
0.9124066679999999
0.912390155
0.912392984
0.912527337
0.912966877
0.913711443
0.9143343909999999
0.9141783170000001
0.913826603
0.9136182840000001
0.913734266
0.9142306809999999
0.91459301
0.914502587
0.913928823
0.913400513
0.913364198
0.913747746
0.914101671
0.914173968
0.913989084
0.9136893
0.9134936509999999
0.913410167
0.913339314
0.913182563
0.91300221
0.9129300159999999
0.912934617
0.912869247
0.9128710959999999
0.9129310279999999
0.9131058540000001
0.9136137609999999
0.914742877
0.916389641
0.918057407
0.91870985
0.918062215
0.9167768940000001
0.91587735
0.916176089
0.917019101
0.9176307570000001
0.9176469540000001
0.917437119
0.917153707
0.916856502
0.916587995
0.9160252190000001
0.9151634809999999
0.914285238
0.913709254
0.9134348609999999
0.9133343209999999
0.9131172940000001
0.91281235
0.9124965859999999
0.91

0.906252252
0.90627941
0.906467226
0.906581352
0.9066658209999999
0.906963319
0.906948035
0.9064212559999999
0.906096067
0.906154755
0.905987201
0.906206431
0.906548591
0.906494762
0.906830218
0.9068295270000001
0.9066616609999999
0.906608165
0.906220268
0.906179022
0.9064128659999999
0.906627022
0.907031228
0.9071428159999999
0.9069417329999999
0.9073066670000001
0.907606349
0.907690049
0.9084036190000001
0.908595107
0.907748386
0.90775503
0.9077172859999999
0.907197754
0.9074423079999999
0.907838844
0.90799831
0.907925227
0.907807928
0.907918978
0.907885435
0.907763388
0.907784523
0.90750861
0.90765737
0.908493089
0.908782548
0.908975733
0.909362627
0.909402765
0.9087941709999999
0.908474795
0.90815211
0.90739512
0.9069446529999999
0.907164225
0.907678792
0.9082768520000001
0.908996502
0.909596947
0.909782106
0.909538622
0.909299868
0.9092559020000001
0.9090385940000001
0.9089022440000001
0.9090510620000001
0.909180642
0.909068491
0.9089455740000001
0.9090809870000001
0.909134461
0.9

1.01279491
1.012710888
1.012423784
1.012267435
1.0120559709999999
1.0118534940000001
1.011717947
1.011544379
1.01121855
1.011118382
1.011133905
1.011111378
1.01122661
1.011258831
1.011232339
1.011119254
1.010986226
1.010799018
1.010593428
1.01029873
1.010153932
1.010038419
1.00990167
1.009898556
1.009737858
1.009396523
1.00922365
1.009119327
1.009046903
1.009002982
1.008952593
1.008791085
1.008561222
1.008560987
1.008569049
1.00847888
1.008401921
1.0083126740000001
1.008091934
1.007998115
1.007914856
1.007920348
1.008049212
1.008114913
1.008230983
1.008390785
1.0086402620000001
1.0086298340000002
1.008755417
1.0089614
1.008995925
1.009173335
1.009504443
1.009618618
1.0099470179999999
1.010551041
1.0108345440000002
1.011008427
1.0112765890000002
1.011342264
1.011543712
1.011980487
1.012287798
1.01249121
1.012776583
1.013091868
1.0133446529999999
1.013561398
1.0139017259999998
1.013997059
1.014173921
1.01445644
1.014525457
1.014617911
1.01471772
1.014759462
1.0147447379999999
1.014629429

...

1.2271364409999999
1.227482781
1.227853384
1.228194559
1.22849883
1.228784608
1.229106222
1.229400534
1.229699884
1.230051398
1.230390885
1.230773515
1.231178712
1.231516442
1.231868196
1.232231655
1.232599973
1.233008058
1.233386851
1.233752915
1.234152238
1.2344803420000001
1.234836477
1.235243616
1.235596823
1.235954141
1.236342646
1.236777499
1.237256367
1.237717256
1.238141852
1.238549952
1.238918217
1.239280882
1.2396433409999998
1.239946637
1.240220291
1.2404864420000001
1.240745629
1.24091918
1.2410534009999998
1.2411761479999999
1.241288338
1.241319402
1.241364446
1.241527024
1.24186094
1.242193268
1.24248132
1.242768119
1.2431808309999999
1.243639955
1.244067478
1.244404544
1.24462653
1.244768613
1.24488018
1.24497212
1.245083087
1.245206164
1.245309488
1.245474769
1.245656264
1.245840528
1.246105713
1.246381625
1.2466708640000002
1.24699458
1.247320332
1.247657392
1.247988174
1.248270077
1.248511334
1.248712931
1.248918567
1.24929085
1.249616083
1.24989197
1.250209318
1.2504

1.379870046
1.381169096
1.379569389
1.387260766
1.38746536
1.3809491109999998
1.3777178819999998
1.37462132
1.373228928
1.3711351230000002
1.3713741190000002
1.3760035519999998
1.376764295
1.37505072
1.3754488390000001
1.3770438280000001
1.376144445
1.375593953
1.373201713
1.3734352459999999
1.37442222
1.375002016
1.378840314
1.378997414
1.376153345
1.3756611840000001
1.375426386
1.3757648919999999
1.3768744240000002
1.377269742
1.378128178
1.377954805
1.376721149
1.376336889
1.3754426640000001
1.3759804619999998
1.378064925
1.378469018
1.379166607
1.3788829980000001
1.376612519
1.376192053
1.37623643
1.3764660919999998
1.3778224940000001
1.378383748
1.378787847
1.3789428209999999
1.3777938930000002
1.378290125
1.380785792
1.381098089
1.380675885
1.3810304919999998
1.380108365
1.379596534
1.3801357930000002
1.3811025730000002
1.380622861
1.3802403269999999
1.380085389
1.379987685
1.380105277
1.38046333
1.3805647030000001
1.380656339
1.380957398
1.381335103
1.381251061
1.381076628000000

1.4613755940000002
1.4622909569999998
1.467799184
1.47071341
1.472101544
1.472534934
1.469525595
1.469955057
1.472612094
1.47149671
1.466167688
1.4563515530000002
1.439819225
1.426172079
1.409824279
1.3978137830000001
1.386131368
1.376831511
1.370108303
1.3648946919999998
1.3600893630000002
1.357419865
1.3595477280000001
1.361099491
1.3617589819999998
1.365762671
1.370558622
1.373853279
1.3787509690000002
1.3869071130000001
1.395520375
1.4025212169999999
1.411321111
1.421918829
1.432672634
1.443163661
1.451668381
1.45626052
1.460671558
1.465762361
1.469305439
1.472370252
1.47635061
1.480201603
1.480596815
1.480341881
1.480604024
1.4805782330000001
1.4800711009999998
1.4789311280000002
1.477410551
1.4730265759999999
1.467909811
1.4640368959999999
1.45772767
1.449768966
1.442456479
1.435039751
1.423814315
1.4161732830000002
1.411052444
1.402026606
1.3949557769999998
1.388591914
1.3785845890000001
1.3719456540000001
1.36724009
1.362651399
1.3596136540000001
1.3563993859999999
1.353911932


0.760280895
0.76024227
0.760177312
0.760105638
0.760117445
0.7601558420000001
0.7601992559999999
0.760246425
0.76025163
0.760262474
0.760281329
0.760235732
0.760203392
0.76019509
0.760208166
0.760258929
0.760277564
0.760270653
0.7602722190000001
0.760216862
0.760184525
0.760200315
0.760221867
0.760321281
0.760406378
0.760406637
0.760358827
0.7603142359999999
0.7602841770000001
0.7602578879999999
0.760211689
0.760148103
0.760099036
0.760111537
0.760179858
0.760259974
0.7603084640000001
0.760293515
0.7602621999999999
0.76026236
0.7603032409999999
0.760282118
0.760283977
0.7603204790000001
0.760367219
0.760409654
0.7604358640000001
0.76043762
0.760419958
0.760469981
0.760528463
0.7605578940000001
0.7605844759999999
0.760628842
0.7606189879999999
0.7605184159999999
0.760443069
0.760426375
0.760464039
0.7605206870000001
0.760528102
0.760544992
0.760584527
0.760609312
0.760577921
0.760546352
0.760545908
0.7605482440000001
0.760556109
0.76054743
0.760531197
0.760625335
0.760687057
0.76070177


0.769816893
0.767618244
0.766833471
0.7662620309999999
0.764428318
0.765855525
0.77415765
0.778087273
0.782520137
0.786448798
0.781257845
0.781240646
0.783323565
0.785793367
0.787301007
0.790705697
0.7971247020000001
0.8027163979999999
0.804034166
0.812472777
0.83388514
0.8350062090000001
0.823206902
0.82231704
0.8128148829999999
0.810853277
0.8302021999999999
0.8297447929999999
0.807691353
0.80653134
0.816702982
0.814272778
0.8177488940000001
0.813020605
0.81017807
0.798571802
0.796706604
0.801632456
0.7992328240000001
0.794685357
0.791784053
0.7889198759999999
0.786022542
0.7834633809999999
0.781259305
0.77798525
0.777461595
0.7800236690000001
0.778132715
0.775179785
0.773312703
0.76966539
0.7678315170000001
0.768069753
0.76704461
0.765078044
0.764178713
0.7637859220000001
0.7630375779999999
0.762655455
0.761980412
0.760898869
0.759976847
0.759040391
0.758294108
0.757899524
0.756933739
0.7565405690000001
0.755955192
0.7551722590000001
0.754128224
0.753616008
0.753194748
0.75263125400

In [6]:
df = pd.read_csv('processed_Data.csv', low_memory=False)

def computeED (df):

    lakes = list(df['lake_name'].unique())
    wavelengths = list(df['wavelength'].unique())

    ed = []
    lref = []
    lake_name = []
    wls = []
    lsw = []
    lsky = []
    
    for lake in lakes:
        lake_df = df.loc[df['lake_name']==lake]
        for wavelength in wavelengths:
            new_df = lake_df.loc[lake_df['wavelength']==wavelength]
            name = new_df['lake_name'].head(1).item()
            wl = new_df['wavelength'].head(1).item()
            sky_df = new_df.loc[new_df['rep_name']=='sky']            
            sky = sky_df['asd_measure'].item()
            reference_df = new_df.loc[new_df['rep_name']=='wr']            
            ref = reference_df['asd_measure'].item()
            downwelling_irradiance = ref * 1 * math.pi
            water_df = new_df.loc[new_df['rep_name']=='water'] 
            water = water_df['asd_measure'].item()
            lsw.append(water)
            lsky.append(sky)
            ed.append(downwelling_irradiance)
            lref.append(ref)
            lake_name.append(name)
            wls.append(wl)
            
    lake_data = {}
    lake_data['ed'] = ed
    lake_data['lake_name'] = lake_name
    lake_data['lref'] = lref
    lake_data['lsky'] = lsky
    lake_data['lsw'] = lsw

    lake_data['wavelength'] = wls
    return pd.DataFrame.from_dict(lake_data)

In [7]:
done = computeED(df)

In [8]:
done.head(2)

,ed,lake_name,lref,lsky,lsw,wavelength
0,1.993682,9mile,0.634609,1.663184,0.046339,350
1,1.991035,9mile,0.633766,1.654474,0.046344,351


### Calculate water-leaving radiance

Downwelling irrandiance ($E_d(\lambda)$) is calculated as: 

$L_w(\lambda)= L_{sw}(\lambda) - \rho_{air -sky}* L_{sky}(\lambda) $  

where: 

- $L_w(\lambda)$ = water-leaving radiance   
- $L_{sw}(\lambda)$ = total radiance measured above water
- $\rho_{air -sky}$ = sun and sky glint correction coefficient at the water surface
- $L_{sky}(\lambda)$ - sky diffuse radiance

and    
$\rho_{air -sky} = \frac{L_{sky}(\lambda=750)}{E_{D}(\lambda=750)} \ge 0.05 \rightarrow 0.0256$

or     

$\rho_{air -sky} = \frac {L_{sky}(\lambda=750} {E_{D}(\lambda=750)} \lt 0.05 \rightarrow 0.0256 + 0.00039W + 0.000034W^2$

where
- W is windspeed $(m s^{-1})$

In [35]:
windspeed = 10 

lw = []

def calculateLW(df):
    for lake in lakes: 
        lake_df = df.loc[df['lake_name']==lake].copy()
        nir = lake_df.loc[lake_df['wavelength']==750].copy()
        #print(nir)
        nir['Lsky_Ed_ratio'] = nir['lsky']/nir['ed']
        
        if (nir['Lsky_Ed_ratio'].iloc[0] >=0.05):
            rho = 0.0256
            #print(rho)
        else: 
            rho = 0.0256 + 0.00039*windspeed + 0.000034*windspeed**2
        
        water_leaving_radiance = lake_df['lsw'] - lake_df['lsky'] *rho
        lw.append(water_leaving_radiance)
    return lw

# not sure the best way to add this back in as a new column to the dataframe

In [41]:
lw = calculateLW(done)
lw[:10]

[0       0.003762
 1       0.003990
 2       0.003794
 3       0.004002
 4       0.004226
 5       0.004198
 6       0.004421
 7       0.004370
 8       0.004315
 9       0.004541
 10      0.004573
 11      0.004683
 12      0.004888
 13      0.005070
 14      0.005106
 15      0.005171
 16      0.005374
 17      0.005718
 18      0.005823
 19      0.005780
 20      0.005752
 21      0.005891
 22      0.005968
 23      0.006045
 24      0.006179
 25      0.006220
 26      0.006260
 27      0.006354
 28      0.006513
 29      0.006610
           ...   
 2121   -0.041686
 2122   -0.027440
 2123   -0.037417
 2124   -0.061681
 2125   -0.113592
 2126   -0.189432
 2127   -0.103007
 2128    0.072794
 2129    0.350867
 2130   -0.794034
 2131   -0.075969
 2132   -0.349535
 2133   -0.189408
 2134   -0.224478
 2135   -0.460748
 2136    0.301499
 2137    0.240759
 2138   -3.788253
 2139   -1.499237
 2140   -0.113568
 2141    0.111359
 2142   -0.065174
 2143   -0.084328
 2144   -0.151453
 2145   -0

### Calculate Remote Sensing Reflectance

Downwelling irrandiance ($E_d(\lambda)$) is calculated as: 

$R_{rs}(\lambda) = \frac{L_w(\lambda)}{E_d(\lambda)}$  

where: 

- $R_{rs}$ = remote sensing reflectance 
- $E_d(\lambda)$ = downwelling irradiance   
- $L_{w}(\lambda)$ = water-leaving radiance  

In [ ]:
def calculateRRS(df):
    rrs = df['Lw']/df['Ed']
    return rrs 

## add back in as a new column to the dataframe